In [2]:
import pandas as pd
import sqlite3
import plotly.express as px
import cv2

# Connexion

In [3]:
connexion = sqlite3.connect("imdb.db")
c = connexion.cursor()

# Analysis
>
> The purpose of this notebook is to analyze the data of the database. There are several categories and we need to understand how they are distributed, how they are linked to our target - which is `grade`. 

## Categories

In [10]:
query = ("SELECT COUNT(Film.id) AS nb_films, Category.category FROM Film "
         "INNER JOIN Category on Category.id == Film.category_id"
         " GROUP BY Category.category;")
cat = pd.read_sql(query, con = connexion)

fig = px.pie(cat, values='nb_films', names='category', title='Proportion of movies by category')
fig.write_image("img/Proportion_of_movies_by_category.png")

> There are 18 categories. It could be an idea to gather in a common category the less popular movies in `Other`.
>
> But before, let's look at how popular they are with the public

In [11]:
query = ("SELECT AVG(Film.grade) AS avg_grade, Category.category FROM Film "
         "INNER JOIN Category on Category.id == Film.category_id "
         "GROUP BY Category.category;")
grade_by_cat = pd.read_sql(query, con = connexion)

fig = px.bar(grade_by_cat, x = 'category', y = 'avg_grade', color='avg_grade', 
             labels={'avg_grade': 'average grade'})
fig.write_image("img/category_grade.png")

> Among the less popular categories, we see that if we put musical and western together, it will create a bias since there is a huge gap between both of them.
>
> It would be tempting to bring `Music` and `Musical` together. In fact, there are only 2 of each, so it is impossible to generalize. **So, we bring the minorities together.**

## Languages and countries
>
> How many languages there are and how they are distributed

In [8]:
lang = c.execute("SELECT COUNT(DISTINCT(language)) FROM Film ").fetchmany()[0][0]

print(f"There are {lang} different languages")

There are 89 different languages


In [12]:
query = "SELECT COUNT(id) AS nb_films, language FROM Film GROUP BY language;"
lang = pd.read_sql_query(query, con = connexion)

fig = px.pie(lang, values='nb_films', names='language', title='Proportion of movies by language')
fig.write_image("img/Proportion_of_movies_by_language.png")

> Which the languages the least represented?

In [21]:
query = ("SELECT COUNT(id) AS nb_films, language FROM Film" 
         " GROUP BY language"
         " HAVING nb_films IN (1,2,3)")

pd.read_sql_query(query, con = connexion).head(10)

,nb_films,language
0,1,None
1,1,basque country
2,1,country estate
3,1,country house
4,1,country music singer
5,1,country singer
6,3,cross country
7,1,cross country running
8,3,cross country trip
9,2,english countryside


> We see that there is a majority of movies in English. **The idea would be to bring them togetherg by continent or common area and get rid of movies with particular languages**.
>
> Let's do the same with the countries

In [11]:
count = c.execute("SELECT COUNT(DISTINCT(country)) FROM Film ").fetchmany()[0][0]

print(f"There are {count} different countries")

There are 83 different countries


In [13]:
query = "SELECT COUNT(id) AS nb_films, country FROM Film GROUP BY country;"
lang = pd.read_sql_query(query, con = connexion)
fig = px.pie(lang, values='nb_films', names='country', title='Proportion of movies by country')
fig.write_image("img/Proportion_of_movies_by_country.png")

> Which country are the least represented?

In [13]:
query = ("SELECT COUNT(id) AS nb_films, country FROM Film" 
         " GROUP BY country"
         " HAVING nb_films IN (1,2,3)")
pd.read_sql_query(query, con = connexion).head(10)

,nb_films,country
0,1,fictional language
1,1,foreign language
2,2,foreign language adaptation
3,1,invented language
4,1,language barrier
5,1,language lesson
6,1,misuse of language
7,1,multiple language version
8,1,portuguese language
9,1,strong language


> **The idea is the same as for the languages**

In [14]:
query = ("SELECT ROUND(AVG(Film.grade),2) AS avg_grade, film.country, COUNT(Film.id) AS nb_films FROM Film "
         "GROUP BY film.country "
         "HAVING nb_films != 1 "
         "ORDER BY avg_grade DESC "
         "LIMIT 15; ")

df = pd.read_sql_query(query, con = connexion)

fig = px.bar(df, x = 'country', y = 'avg_grade', color='avg_grade', hover_data=['nb_films'], 
             labels={'avg_grade': 'average grade'})
fig.write_image("img/country_grade.png")

> We notice that the most well rated movies are from the least represented countries - excepted for South Korea and Japan with respectively 75 and 131 movies. This could introduce an overfitting in the future because it is impossible to generalize properly with so few movies. 
>
> **The idea to bring together by continent seems to help to counter the problem because it allows to get a mean, much more representative of the reality**
>
> Now, it is important to understand the distribution of actors and directors over the database. 

## Actors and directors

### Actors

In [15]:
query = ("SELECT Actor.actor, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Actor ON Casting.actor_id == Actor.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Actor.actor "
         "HAVING nb_films > 20 "
         "ORDER BY avg_grade DESC, nb_films DESC")

df = pd.read_sql_query(query, con = connexion)

fig = px.bar(df, x = 'actor', y = 'nb_films', color = 'avg_grade', labels={'nb_films': 'number of movies'})
fig.write_image("img/actor_movie_grade.png")

> It is important to keep some names because it impacts the grade and allows to generalize as much as possible this one. The actors with the best (worst) average grade are those with only one movie to their credit as shown below.

In [16]:
query = ("SELECT Actor.actor, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Actor ON Casting.actor_id == Actor.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Actor.actor "
         "ORDER BY avg_grade DESC, nb_films DESC ")

pd.read_sql_query(query, con = connexion).head()

,actor,nb_films,avg_grade
0,Tanner Buchanan,1,8.8
1,Kathir,1,8.7
2,Natasa Ninkovic,1,8.7
3,Alexandre Rodrigues,1,8.6
4,Vijay Sethupathi,1,8.6


In [17]:
query = ("SELECT Actor.actor, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Actor ON Casting.actor_id == Actor.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Actor.actor "
         "ORDER BY avg_grade ASC, nb_films DESC ")

pd.read_sql_query(query, con = connexion).head()

,actor,nb_films,avg_grade
0,Paris Hilton,2,1.85
1,Anil Kapoor,1,1.90
2,Carmen Electra,1,1.90
3,Jon Voight,1,1.90
4,Brian Austin Green,1,2.00


> Hopping to generalize as much as possible, **we will keep the name of the top 17 of well-known actors and put all the others together in a common category : `Other` - because it is impossible to generalize from them.**

### Directors

In [16]:
query = ("SELECT Director.director, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Director ON Casting.director_id == Director.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Director.director "
         "HAVING nb_films > 10 "
         "ORDER BY avg_grade DESC, nb_films DESC")

df = pd.read_sql_query(query, con = connexion)

fig = px.bar(df, x = 'director', y = 'nb_films', color = 'avg_grade', labels={'nb_films': 'number of movies'})
fig.write_image("img/director_movie_grade.png")

In [19]:
query = ("SELECT Director.director, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Director ON Casting.director_id == Director.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Director.director "
         "ORDER BY avg_grade DESC, nb_films DESC ")

pd.read_sql_query(query, con = connexion).head()

,director,nb_films,avg_grade
0,John B. Crye,1,8.8
1,Mari Selvaraj,1,8.7
2,Predrag Antonijevic,1,8.7
3,C. Prem Kumar,1,8.6
4,Vetrimaaran,2,8.5


In [20]:
query = ("SELECT Director.director, COUNT(Film.id) AS nb_films, ROUND(AVG(Film.grade),2) as avg_grade FROM Casting "
         "INNER JOIN Director ON Casting.director_id == Director.id "
         "INNER JOIN Film ON Film.id == Casting.id_title "
         "GROUP BY Director.director "
         "ORDER BY avg_grade ASC, nb_films DESC ")

pd.read_sql_query(query, con = connexion).head()

,director,nb_films,avg_grade
0,Darren Doane,1,1.4
1,Lawrence Kasanoff,1,1.5
2,William Heins,1,1.8
3,Bob Clark,1,1.9
4,Tom Putnam,1,1.9


> **Same observation with the directors**